In [1]:
# importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Mounting the google drive

from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
# Loading the data in two Pandas dataframes

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/MRP/creditcard.csv')

In [4]:
# We split it into X_train, X_test, y_train, y_test for the purpose of classification_report and confusion_matrix.

from sklearn.model_selection import train_test_split

X = data.drop('Class', axis = 1).values
y = data['Class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=101)

In [5]:
# We split the data agian here in two dataframes; legitimate and anomaly. Then we drop the 'Class' column from out original 'data' 
# and take all the legitimate transactions in the data_legitimate variable and the anomalies in the data_anomaly variable.
# That is why this is unsupervised.

legitimate = data['Class'] == 0
anomaly = data['Class'] != 0

data.drop ('Class', axis = 1, inplace = True)
data_legitimate = data[legitimate]
data_anomaly = data[anomaly]

data_legitimate_train, data_legitimate_test = train_test_split(data_legitimate, test_size = 0.2, random_state=101)

print(f"Legitimate Count: {len(data_legitimate)}")
print(f"Anomaly Count: {len(data_anomaly)}")

print(f"Legitimate train Count: {len(data_legitimate_train)}")
print(f"Legitimate test Count: {len(data_legitimate_test)}")

Legitimate Count: 284315
Anomaly Count: 492
Legitimate train Count: 227452
Legitimate test Count: 56863


In [6]:
# importing the necessary libraries

import tensorflow as tf
from keras import models, layers
from keras.layers import Dense
from keras.models import Sequential

In [16]:
# defnining the autoencoder and compiling the model and fitting the model with data_legitimate_train only

model = Sequential()
model.add(Dense(50, input_dim = data_legitimate_train.shape[1], activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(data_legitimate_train.shape[1]))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(data_legitimate_train, data_legitimate_train, verbose = 1, epochs = 200)

Epoch 1/200
7108/7108 [==============================] - 19s 3ms/step - loss: 1647055.0000
Epoch 2/200
7108/7108 [==============================] - 19s 3ms/step - loss: 925.9426
Epoch 3/200
7108/7108 [==============================] - 19s 3ms/step - loss: 922.4399
Epoch 4/200
7108/7108 [==============================] - 19s 3ms/step - loss: 703.9388
Epoch 5/200
7108/7108 [==============================] - 18s 3ms/step - loss: 778.3711
Epoch 6/200
7108/7108 [==============================] - 19s 3ms/step - loss: 651.5154
Epoch 7/200
7108/7108 [==============================] - 20s 3ms/step - loss: 667.6955
Epoch 8/200
7108/7108 [==============================] - 20s 3ms/step - loss: 732.8787
Epoch 9/200
7108/7108 [==============================] - 19s 3ms/step - loss: 525.0688
Epoch 10/200
7108/7108 [==============================] - 19s 3ms/step - loss: 634.1699
Epoch 11/200
7108/7108 [==============================] - 19s 3ms/step - loss: 554.4779
Epoch 12/200
7108/7108 [=============

In [17]:
# The autoencoder model summary

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                1550      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 255       
_________________________________________________________________
dense_6 (Dense)              (None, 50)                300       
_________________________________________________________________
dense_7 (Dense)              (None, 30)                1530      
Total params: 3,635
Trainable params: 3,635
Non-trainable params: 0
_________________________________________________________________


In [18]:
# The classification report for the autoencoder

from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict_classes(X_test)
predictions = np.where(predictions > 0, 1, 0)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56859
           1       0.00      0.00      0.00       103

    accuracy                           1.00     56962
   macro avg       0.50      0.50      0.50     56962
weighted avg       1.00      1.00      1.00     56962



In [19]:
# The consusion matrix for the autoencoder

confusion_matrix(y_test,predictions)

array([[56836,    23],
       [  103,     0]])

In [20]:
# The RMSE score for legitimate and anomaly section of the whole data

from sklearn import metrics

pred = model.predict(data_legitimate_test)
score1 = np.sqrt(metrics.mean_squared_error(pred, data_legitimate_test))
print(f"data_legitimate_test Score (RMSE): {score1}".format(score1))

pred = model.predict(data_legitimate)
score2 = np.sqrt(metrics.mean_squared_error(pred, data_legitimate))
print(f"data_legitimate Score (RMSE): {score2}".format(score2))

pred = model.predict(data_anomaly)
score3 = np.sqrt(metrics.mean_squared_error(pred, data_anomaly))
print(f"anomaly Score (RMSE): {score3}".format(score3))

data_legitimate_test Score (RMSE): 4.631036309006115
data_legitimate Score (RMSE): 4.65557769336348
anomaly Score (RMSE): 6.411216224052135
